In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import pickle
from pathlib import Path
from kuma_utils.utils import sigmoid
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import Ridge

from datasets import *
from transforms import *
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

In [3]:
test = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/sample_submission.csv')
valid = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/v18v.csv')
public748 = pd.read_csv('input/public748.csv')
iafoss_v5160 = {
    'oof': pd.read_csv('input/predictions/val_model_v5_1_6_0.csv'),
    'pred': pd.read_csv('input/predictions/submission_model_v5_1_6_0.csv')}
drhb_exp200 = {
    'oof': pd.read_csv('input/predictions/EXP_200_BASELINE_CASHE_V4_EVAL.csv', usecols=['id', 'pred']),
    'pred': pd.read_csv('input/predictions/EXP_200_BASELINE_CASHE_V4.csv')}
# drhb_exp200['oof'].rename({'pred': 'drhb_exp200'}, axis=1, inplace=True)

In [5]:
model_paths = [
    # Path('results/ds_09_val0'),
    Path('results/ds_09_val1'),
    Path('results/aug_04'),
    Path('results/ds_14'),
    Path('results/ds_15'),
    Path('results/ds_16')
]


In [9]:
prediction_df = {}
oof_df = {}
for p in model_paths:
    pred = sigmoid(np.load(p/'predictions_tta.npy'))
    oof = sigmoid(np.load(p/'outoffolds_tta.npy'))

    if pred.shape[1] == 1:
        pred = pred.reshape(-1)
    elif pred.shape[0] == 5:
        pred = pred[0].reshape(-1)
    prediction_df[p.stem] = pred
    oof_df[p.stem] = oof.reshape(-1)
prediction_df = pd.DataFrame(prediction_df)
prediction_df['iafoss_v5160'] = iafoss_v5160['pred']['target']
prediction_df['drhb_exp200'] = drhb_exp200['pred']['target']
oof_df = pd.DataFrame(oof_df)
oof_df['id'] = valid['id']
oof_df['iafoss_v5160'] = iafoss_v5160['oof']['target']
oof_df['drhb_exp200'] = drhb_exp200['oof']['pred']
# oof_df = oof_df.merge(drhb_exp200['oof'].drop(['path', 'base_id'], axis=1), on='id', how='left').drop('id', axis=1)
# prediction_df['public748'] = public748['target'].values

In [10]:
prediction_df

,ds_09_val1,aug_04,ds_14,ds_15,ds_16,iafoss_v5160,drhb_exp200
0,0.378731,0.345557,0.328906,0.506897,0.536705,0.507324,0.356428
1,0.968598,0.984320,0.994552,0.978992,0.987508,0.951172,0.999973
2,0.337151,0.398930,0.292780,0.458657,0.415963,0.445312,0.268465
3,0.439851,0.539125,0.396398,0.507958,0.625136,0.693848,0.779437
4,0.663693,0.482278,0.654054,0.599116,0.690989,0.983887,0.997136
...,...,...,...,...,...,...,...
7970,0.999416,0.998426,0.999989,0.830332,0.993844,1.000000,0.999996
7971,0.486552,0.544869,0.434396,0.558710,0.540350,0.505371,0.420342
7972,0.262954,0.328540,0.262360,0.460538,0.425600,0.706543,0.213687
7973,0.448427,0.584629,0.329145,0.600259,0.455129,0.553711,0.271413


In [11]:
oof_df['target'] = valid['target']
oof_df.dropna(inplace=True)
oof_df

,ds_09_val1,aug_04,ds_14,ds_15,ds_16,id,iafoss_v5160,drhb_exp200,target
0,0.360660,0.389735,0.289726,0.459198,0.416431,3c451363c_negative,0.521484,0.270715,0
1,0.345279,0.389357,0.307953,0.466254,0.413770,d036cb94b_negative,0.471680,0.308455,0
2,0.408482,0.501199,0.410116,0.509803,0.530247,ef4bfb410_negative,0.513184,0.272621,0
3,0.378056,0.443244,0.257842,0.505781,0.446430,c374ae4ef_negative,0.532715,0.320555,0
4,0.509641,0.532739,0.349308,0.520299,0.494942,48355bd69_negative,0.597656,0.808276,0
...,...,...,...,...,...,...,...,...,...
7970,0.322120,0.396789,0.292915,0.465174,0.575605,ffbce04ef_weak,0.470703,0.248373,1
7971,0.381412,0.382718,0.294402,0.492523,0.438692,ffc2d976b_weak,0.556641,0.678827,1
7972,0.609443,0.788830,0.476767,0.561197,0.550176,ffc905909_weak,0.998535,0.926371,1
7973,0.386961,0.415397,0.352287,0.489313,0.493704,ffe276f3e_weak,0.531738,0.315376,1


In [13]:
model = Ridge(positive=True)
X = oof_df.drop(['target', 'id'], axis=1)
y = oof_df['target']
X_test = prediction_df
model.fit(X, y)
y_pred = model.predict(X)
y_stack = model.predict(X_test)
print('stacking cv', roc_auc_score(y, y_pred))
print('weight', model.coef_)

stacking cv 0.7503985638248079
weight [0.09468512 0.08156054 0.         0.         0.14765501 0.66952851
 0.        ]


In [14]:
y_stack

array([0.50301967, 0.97470113, 0.44408901, ..., 0.6076468 , 0.54812992,
       0.48485966])

In [12]:
# test['target'] = prediction_df['aug_04'] * 0.2 + prediction_df['ds_05_aug2'] * 0.2 + prediction_df['public748'] * 0.6
# test['target'] = prediction_df['ds_09'] * 0.33 + prediction_df['ds_13'] * 0.33 + prediction_df['public748'] * 0.33
# test['target'] = prediction_df['ds_15']
test['target'] = y_stack

In [14]:
test

,id,target
0,00054c878,0.494353
1,0007285a3,0.968023
2,00076c5a6,0.444898
3,001349290,0.659862
4,001a52e92,0.908924
...,...,...
7970,ffbce04ef,1.009675
7971,ffc2d976b,0.525043
7972,ffc905909,0.614815
7973,ffe276f3e,0.547437


In [15]:
test.to_csv('results/stacking_submission_iafossv5160_drhbexp200_ds19val0_aug04_ds14_ds15.csv', index=False)
# test.to_csv(Path('results/ds_15')/'submission.csv', index=False)